In [ ]:
import os
import sys
import glob
import re
import subprocess
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
%matplotlib inline
os.chdir('/local/home/dhaziza/entrack')
sys.path.append('/local/home/dhaziza/entrack/')

from src.data.mri_pipeline import MriPreprocessingPipeline
from src.data.plots import display_image_path

ppmi_data = '/local/PPMI'
files_glob = '/local/PPMI/raw/*/*/*/*/*.nii'
extract_image_id_regexp = '.*S(\d+)_I(\d+)\.*'


# Helper functions
def intvals(s):
    vals = re.findall(r'\d+', s)
    return [int(i) for i in vals]

def intval(s):
    vals = intvals(s)
    return vals[0]

# Fill a map from image_id to actual image file
def load_images_store():
    all_images = {}
    regexp = re.compile(extract_image_id_regexp)
    for f in glob.glob(files_glob):
        m = regexp.match(f)
        if m is None:
            print('/!\ No regexp match for %s' % f)
        f_image_id = int(m.group(2))
        all_images[f_image_id] = {'path': f}
    print('Loaded %s image paths' % len(all_images))

    # Fill XML data
    def elem_unique(path):
        e = root.findall(path)
        assert(len(e) == 1)
        return e[0]
    xml_without_images = []
    loaded_ok = 0
    for f in glob.glob('%s/raw/*_I*.xml' % (ppmi_data)):
        m = regexp.match(f)
        if m is None:
            print('/!\ No regexp match for %s' % f)
        f_image_id = int(m.group(2))
        if f_image_id not in all_images:
            xml_without_images.append(f)
            continue
        img_values = all_images[f_image_id]

        tree = ET.parse(f)
        root = tree.getroot()
        img_values['flip_angle'] = intval(elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Flip Angle']").text)
        img_values['acquisition_plane'] = elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Acquisition Plane']").text
        img_values['weighting'] = elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Weighting']").text
        img_values['shape'] = (
            intval(elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Matrix X']").text),
            intval(elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Matrix Y']").text),
            intval(elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Matrix Z']").text),
        )
        img_values['shape_from_file'] = nib.load(img_values['path']).get_data().shape
        loaded_ok += 1
    print('Loaded %d xml files (minus %d without associated .nii file)' % (loaded_ok, len(xml_without_images)))
    return all_images

images_store = load_images_store()

In [ ]:
def debug_display_img(image_id):
    if image_id not in images_store:
        return
    path = images_store[image_id]['path']
    img = nib.load(path)
    dat = img.get_data()
    display_image_path(path)
    #display_image_path('%s/01_brain_extracted/I%s.nii.gz' % (ppmi_data, image_id), False)
    #display_image_path('%s/02_registered/I%s.nii.gz' % (ppmi_data, image_id), False)

def print_orientation_from_xml(image_id):
    img_data = images_store[image_id]
    print('%s (FLIP %s / %s)' % (img_data['acquisition_plane'], img_data['flip_angle'], img_data['weighting']))

In [ ]:
#display_image_path('/local/home/dhaziza/entrack/data/raw/KOLN_T1/3362P/111202/3362_t1.nii.gz')
#display_image_path('/local/fsl/data/standard/MNI152_T1_0.5mm.nii.gz')

def ppmi_image_debug_orient(image_id):
    img_data = images_store[image_id]
    print('#############################################""')
    print(image_id)
    print(img_data['path'])
    print(img_data['shape'])
    print_orientation_from_xml(image_id)
    debug_display_img(image_id)

images_list = [
    (dat['acquisition_plane'], dat['flip_angle'], id)
    for id, dat in images_store.items()
    if dat['shape'][2] > 60 and dat['shape'][0] < 800 and dat['weighting'] == 'T1'
]
images_list.sort()
images_list = [e[2] for e in images_list]
print('Found and sorted %s MRI' % (len(images_list)))
images_list = images_list[::len(images_list)/20]
for image_id in images_list:
    ppmi_image_debug_orient(image_id)

In [ ]:
import random

t2_sizes = [
    np.array(dat['shape_from_file'])[0:3]
    for id, dat in images_store.items()
    if dat['weighting'] == 'T2'
]
t2_size_with_norm = [
    s + np.random.normal(scale=8.0, size=3)
    for s in t2_sizes
]
print('T2 images: %d' % len(t2_sizes))
t2_sizes_x = np.array([s[0] for s in t2_size_with_norm])
t2_sizes_y = np.array([s[1] for s in t2_size_with_norm])
t2_sizes_z = np.array([s[2] for s in t2_size_with_norm])
plt.scatter(t2_sizes_x, t2_sizes_y, s=1)
plt.xlabel('x')
plt.ylabel('y')
plt.title('x / y shape of PPMI T2 images')
plt.show()
plt.scatter(t2_sizes_x, t2_sizes_z, s=1)
plt.xlabel('x')
plt.ylabel('z')
plt.title('x / z shape of PPMI T2 images')
plt.show()
plt.scatter(t2_sizes_y, t2_sizes_z, s=1)
plt.xlabel('y')
plt.ylabel('z')
plt.title('y / z shape of PPMI T2 images')
plt.show()